# Inference

This script showcases the different models available in fishbAIT and how to use them efficiently.

In [1]:
import torch
import pandas as pd
import numpy as np
from inference_utils.fishbAIT_for_inference import fishbAIT_for_inference

Specify the model version and load the model

In [2]:
MODEL_TYPE = 'EC50EC10'
SPECIES_GROUP = 'fish'
MODEL_VERSION = f'{MODEL_TYPE}_{SPECIES_GROUP}'

In [3]:
fishbait = fishbAIT_for_inference(model_version=MODEL_VERSION)
fishbait.load_fine_tuned_model()

Downloading:   0%|          | 0.00/722 [00:00<?, ?B/s]

c:\Users\s30578\AppData\Local\miniconda3\envs\fishbait\lib\site-packages\huggingface_hub-0.11.1-py3.8.egg\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\s30578\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


Downloading:   0%|          | 0.00/334M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/149k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/101k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Load the SMILES you wish to predict

In [4]:
data = pd.read_excel('../data/tutorials/Inference_example_2.xlsx')
data

,SMILES,cmpdname
0,CC(=O)Oc1ccccc1C(O)=O,Aspirin
1,[Cr],Chromium
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,Fluoxetine hydrochloride
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,Clofenotane
4,[Cu],Copper
...,...,...
995,[Pb++].[O-]c1c(cc(c([O-])c1[N+]([O-])=O)[N+]([...,Lead styphnate
996,CC(C)(C)C(O)(CCc1ccc(Cl)cc1)Cn2cncn2,Tebuconazole
997,[Na+].[Na+].[Na+].[Na+].OCCN(CCO)c1nc(Nc2ccc(c...,OpticalBrightenerBbu220
998,CNC.OC(=O)COc1ccc(Cl)cc1Cl,"2,4-D dimethylamine salt"


Specify the endpoint and effect you wish to predict and make the prediction

In [5]:
PREDICTION_ENDPOINT = 'EC50'
PREDICTION_EFFECT = 'MOR'
EXPOSURE_DURATION = 96
SMILES_COLUMN_NAME = 'SMILES'

In [6]:
results = fishbait.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].iloc[0:10].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, return_cls_embeddings=True)
results

Renamed NOEC *EC10* in 0 positions


100%|██████████| 2/2 [00:00<00:00,  6.64it/s]


,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC50,MOR,CC(=O)Oc1ccccc1C(=O)O,1.906641,80.656868,"[0.0849871039390564, 0.7132208347320557, 0.520..."
1,[Cr],1.982271,EC50,MOR,[Cr],1.762321,57.852364,"[-0.500830888748169, 0.3293803334236145, -0.31..."
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC50,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-0.073505,0.844297,"[-0.15795661509037018, -0.1591150015592575, 0...."
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC50,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-1.696048,0.020135,"[-0.9377384185791016, -1.693954348564148, -0.5..."
4,[Cu],1.982271,EC50,MOR,[Cu],-0.502600,0.314340,"[-1.3534588813781738, 0.17010436952114105, -1...."
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC50,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,1.375471,23.739494,"[-0.19239084422588348, 0.2170313000679016, -0...."
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC50,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,2.412714,258.650909,"[-0.14244793355464935, 0.6918228268623352, -0...."
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC50,MOR,CC(Br)(CO)[N+](=O)[O-],0.747496,5.591080,"[-1.176247239112854, -0.013733251951634884, 0...."
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC50,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.336293,0.046101,"[-1.1987366676330566, -1.1545050144195557, -1...."
9,[Cl-].[Cl-].[Zn++],1.982271,EC50,MOR,[Cl-].[Cl-].[Zn+2],0.197087,1.574298,"[-0.9854727387428284, 0.3889364004135132, -0.5..."


If you run into truble using this model use the `__help__` function

In [ ]:
fishbait.__help__()

# Check the predictions compared to our training sets

In [7]:
from inference_utils.plots_for_space import PlotPCA_CLSProjection
from inference_utils.pytorch_data_utils import check_closest_chemical, check_training_data

Check if chemicals are present in training data. They may be present as either an:
- 'endpoint_match' i.e. the chemical was used for training this model for this species and endpoint.
- 'effect_match' i.e. the chemical was used for training this model for this species, endpoint and effect.

In [8]:
results = check_training_data(results, model_type=MODEL_TYPE, species_group=SPECIES_GROUP, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT)
results

,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC50,MOR,CC(=O)Oc1ccccc1C(=O)O,1.906641,80.656868,"[0.0849871039390564, 0.7132208347320557, 0.520...",0,0
1,[Cr],1.982271,EC50,MOR,[Cr],1.762321,57.852364,"[-0.500830888748169, 0.3293803334236145, -0.31...",1,1
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC50,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-0.073505,0.844297,"[-0.15795661509037018, -0.1591150015592575, 0....",1,1
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC50,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-1.696048,0.020135,"[-0.9377384185791016, -1.693954348564148, -0.5...",1,1
4,[Cu],1.982271,EC50,MOR,[Cu],-0.502600,0.314340,"[-1.3534588813781738, 0.17010436952114105, -1....",1,1
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC50,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,1.375471,23.739494,"[-0.19239084422588348, 0.2170313000679016, -0....",1,1
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC50,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,2.412714,258.650909,"[-0.14244793355464935, 0.6918228268623352, -0....",1,1
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC50,MOR,CC(Br)(CO)[N+](=O)[O-],0.747496,5.591080,"[-1.176247239112854, -0.013733251951634884, 0....",0,0
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC50,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.336293,0.046101,"[-1.1987366676330566, -1.1545050144195557, -1....",1,1
9,[Cl-].[Cl-].[Zn++],1.982271,EC50,MOR,[Cl-].[Cl-].[Zn+2],0.197087,1.574298,"[-0.9854727387428284, 0.3889364004135132, -0.5...",1,1


Next we check if which chemical is closest to the predicted chemicals by evaluating the CLS-embeddings against the training set's CLS-embedding by means of their cosine-similarity:
- cosine-similarity=1 --> Identical structures
- cosine-similarity=-1 --> completely oposite in terms of toxicity

In [9]:
results = check_closest_chemical(results=results, MODELTYPE=MODEL_TYPE, PREDICTION_SPECIES=SPECIES_GROUP, PREDICTION_ENDPOINT=PREDICTION_ENDPOINT, PREDICTION_EFFECT=PREDICTION_EFFECT)
results

,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match,most similar chemical,cosine similarity
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC50,MOR,CC(=O)Oc1ccccc1C(=O)O,1.906641,80.656868,"[0.0849871039390564, 0.7132208347320557, 0.520...",0,0,CC(=O)Oc1ccccc1C(=O)O,1.0
1,[Cr],1.982271,EC50,MOR,[Cr],1.762321,57.852364,"[-0.500830888748169, 0.3293803334236145, -0.31...",1,1,[Cr],1.0
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC50,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-0.073505,0.844297,"[-0.15795661509037018, -0.1591150015592575, 0....",1,1,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],1.0
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC50,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-1.696048,0.020135,"[-0.9377384185791016, -1.693954348564148, -0.5...",1,1,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,1.0
4,[Cu],1.982271,EC50,MOR,[Cu],-0.502600,0.314340,"[-1.3534588813781738, 0.17010436952114105, -1....",1,1,[Cu],1.0
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC50,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,1.375471,23.739494,"[-0.19239084422588348, 0.2170313000679016, -0....",1,1,CCNc1nc(Cl)nc(NC(C)C)n1,1.0
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC50,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,2.412714,258.650909,"[-0.14244793355464935, 0.6918228268623352, -0....",1,1,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,1.0
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC50,MOR,CC(Br)(CO)[N+](=O)[O-],0.747496,5.591080,"[-1.176247239112854, -0.013733251951634884, 0....",0,0,CC(Br)(CO)[N+](=O)[O-],1.0
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC50,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.336293,0.046101,"[-1.1987366676330566, -1.1545050144195557, -1....",1,1,c1ccc2c(c1)-c1cccc3cccc-2c13,1.0
9,[Cl-].[Cl-].[Zn++],1.982271,EC50,MOR,[Cl-].[Cl-].[Zn+2],0.197087,1.574298,"[-0.9854727387428284, 0.3889364004135132, -0.5...",1,1,[Cl-].[Cl-].[Zn+2],1.0


# Plot PCA for all chemicals

Finally we can plot the chemical space built during training of the Transformer module in the model. The space is built by the CLS-embedidngs present in the training set of the model but can be used to project new chemicals onto. The space prepared in this example uses `show_all_predictions=True` which plots additional SMILES, not included in the training data to add interpretability. We also use `inference_df=results` to plot the predicted SMILES from above into the space, however this can be set to `None` if not desired.

The plot can be saved as interactive HTML by `fig.write_html(figurename.html')`

Note that in the hover text of each point, the L1Error from our 10x10-fold cross-validation is included from when the used model was evaluated on that chemical.

In [10]:
from inference_utils.plots_for_space import PlotPCA_CLSProjection

In [11]:
PlotPCA_CLSProjection(model_type=MODEL_TYPE, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, species_group=SPECIES_GROUP, show_all_predictions=True, inference_df=results)

c:\Users\s30578\OneDrive - Semcon AB\fishbAIT\tutorials\inference_utils\plots_for_space.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_preds['pc1'], all_preds['pc2'] = pcac[:len(all_preds),0], pcac[:len(all_preds),1]
c:\Users\s30578\OneDrive - Semcon AB\fishbAIT\tutorials\inference_utils\plots_for_space.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_preds['pc1'], all_preds['pc2'] = pcac[:len(all_preds),0], pcac[:len(all_preds),1]
